# Interpolation module

The Midgard **math.spatial_interpolation** module provides functionality to carry out one dimensional (1D) interpolation. (https://docs.scipy.org/doc/scipy/tutorial/interpolate/ND_unstructured.html#d-example)

## Use interpolation module

An example is shown, how to use the `math.interpolation` module, which includes several interpolation method. The interpolation methods are based on **SciPy** functions, except for `lagrange` interpolation method.

In [ ]:
# Import interpolation module
from midgard.math import interpolation

# Write list with available interpolators
interpolation.interpolators()

In [ ]:
# Third party import
import numpy as np

# Import matplotlib module
import matplotlib.pyplot as plt

# Generate input data for interpolation, whereby x and y represent the original data and x_new the data points 
# for interpolation
x = np.linspace(0, 10, 10)  
y = np.sin(x)
x_new = np.linspace(0, 10, 101)

# Interpolation for given x_new data points and chosen kind of interpolation method
y_new = interpolation.interpolate(x, y, x_new, kind="linear")

# Plot interpolation results
plt.plot(x, y, label="data", linestyle="None", color="blue", marker="o")
plt.plot(x_new, y_new, label="interpolated", color="green")
plt.plot(x_new, np.sin(x_new), label="truth", linewidth=3.0, color="black", alpha=0.2)
plt.title(f"RMS of (truth - interpolated): {np.sqrt(np.nanmean(np.square(np.sin(x_new) - y_new))):.4f}")
plt.legend()
plt.show()

In [ ]:
# Determine the execution time of interpolation method with %timeit
#
# Note: %timeit is a IPython function and can be used to determine the excecution time of statement or expression,
#       whereby 
#          -n: number of loops
#          -r: number of repeats
#
%timeit -n 10 -r 1 y_new = interpolation.interpolate(x, y, x_new, kind="linear")